# E91 – Ein interaktives Jupyter Notebook Tutorial

In diesem interaktiven Tutorial lernst du Schritt für Schritt:
1. **Wie der E91 Algorithmus funktioniert**  
2. **Wie die CHSH-Ungleichung funktioniert**  
3. **Die Unterschiede zum BB84 Algorithmus**  
4. **Wie du mit Qiskit und Bloch-Sphären erste Experimente zur Verschränkung durchführen kannst**  

---

## Voraussetzungen
- Installiertes **Python 3** (empfohlen: ab Version 3.8)
- Installiertes **Qiskit** (mindestens `qiskit` Version 0.30+)
- Grundlegende Kenntnisse zur Quantenmechanik (Qubits, Verschränkung)
- Grundlegende Kenntnisse zum Arbeiten in Jupyter Notebooks

---

## 1. Setup

Führe diese Zelle aus, um sicherzustellen, dass alle benötigten Bibliotheken installiert und importiert sind.

In [ ]:
# Dies ist eine Code-Zelle im Jupyter Notebook
# Installiere Qiskit (falls noch nicht geschehen)
# !pip install qiskit

from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import numpy as np
import matplotlib.pyplot as plt

# Inline Plot in Jupyter
%matplotlib inline

# Simulator für Zustände (Statevector-Simulator)
state_simulator = Aer.get_backend('statevector_simulator')

# Simulator für Messungen (shot-based)
qasm_simulator = Aer.get_backend('qasm_simulator')

print("Setup abgeschlossen. Qiskit und Matplotlib sind importiert.")

## 2. Kurze Wiederholung: Was ist E91?

Der **E91-Algorithmus** (Artur Ekert, 1991) ist ein **entanglement-basiertes** Quantenkryptographie-Protokoll. Das Ziel ist es, zwischen zwei Parteien (Alice und Bob) einen **sicheren Schlüssel** auszutauschen. Die Sicherheit wird über **Verschränkung** und die **Verletzung der Bell- bzw. CHSH-Ungleichung** garantiert.

### Grundidee:
1. **Verschränkte Photonenpaare** werden erzeugt. Jeweils eines der beiden Photonen geht an Alice, das andere an Bob.  
2. Alice und Bob messen ihre Photonen in verschiedenen Basen.  
3. Durch die **Nicht-Lokalität** (CHSH-Verletzung) können sie sicherstellen, dass kein Abhörversuch unbemerkt bleibt.  
4. Stimmt die gemessene Korrelation mit dem erwarteten quantenmechanischen Wert überein, wird ein Großteil der Messwerte verworfen und der Rest dient als **geheimer Schlüssel**.

---

## 3. E91 Algorithmus Schritt für Schritt

### 3.1. Erzeugung und Verteilung verschränkter Zustände
Meistens wird ein **Singulett-Zustand** verwendet, z. B.:
$$\vert\Psi^-\rangle = \frac{1}{\sqrt{2}} (\vert01\rangle - \vert10\rangle).$$

In Qiskit können wir diesen Zustand durch eine einfache Schaltung erzeugen:

In [ ]:
# Erzeugung eines singuletten Zustandes in Qiskit

# Quanten-Register mit 2 Qubits
qc_singlet = QuantumCircuit(2)

# Erzeuge Bell-Zustand (z.B. |Φ+>), den wir dann leicht in |Ψ−> umwandeln können
qc_singlet.h(0)          # Hadamard-Gatter auf Qubit 0
qc_singlet.cx(0, 1)      # CNOT: steuert Qubit 0, Ziel: Qubit 1

# Aktuell ist das der Zustand (|00> + |11>) / sqrt(2) = |Φ+>
# Um daraus |Ψ−> zu machen (|01> - |10>) / sqrt(2):
# Wir können z.B. ein X-Gate auf das zweite Qubit und danach ein Z-Gate auf das zweite Qubit anwenden.
qc_singlet.x(1)
qc_singlet.z(1)

# Schau dir den finalen Zustand an
final_state = execute(qc_singlet, state_simulator).result().get_statevector()
print("Zustandsvektor:", final_state)

# Visualisierung auf Bloch-Sphären (für beide Qubits)
plot_bloch_multivector(final_state)

**Erklärung**:  
- `h(0)` + `cx(0,1)` erzeugt einen Bell-Zustand (Speziell $|\Phi^+\rangle$).  
- Mit `x(1)` und `z(1)` transformieren wir ihn in $|\Psi^-\rangle$.  
- Der Singulett-Zustand ($|\Psi^-\rangle$) ist ein maximals verschränkter Zustand von zwei Qubits.

---

### 3.2. Messung in unterschiedlichen Basen
Im E91-Protokoll messen Alice und Bob ihre Qubits (Photonen) in **zufälligen** Basen, beispielsweise:
- **Basis A**: Z-Basis (Standard-Basis)  
- **Basis A'**: X-Basis (Hadamard-Basis)  
- **Basis B**: Eine andere, z. B. um 45° rotiert  
- **Basis B'**: Noch eine weitere Variante

Im klassischen E91-Protokoll werden mindestens drei Basen verwendet, z. B.  
- Alice wählt aus $\{Z, X, Y\}$  
- Bob wählt aus $\{Z, X, Y\}$

Zur Demonstration nutzen wir hier exemplarisch 2 oder 3 Messbasen.

**Beispiel: Messung in Z-Basis vs. X-Basis**

In [ ]:
def measure_in_basis(circuit, qubit, basis_label='Z'):
    """
    Misst ein gegebenes Qubit in einer bestimmten Basis.
    basis_label kann 'Z' oder 'X' sein.
    """
    if basis_label == 'X':
        # Um in X-Basis zu messen, wende H an, dann Z-Basis Messung
        circuit.h(qubit)
    # Z-Basis Messung (Standard in Qiskit)
    circuit.measure(qubit, qubit)
    return circuit

# Beispiel: Wir messen 1000 Mal
# Qubit 0 (Alice) in Z-Basis,
# Qubit 1 (Bob) in X-Basis
shots = 1000
counts_all = []

for basis_bob in ['Z', 'X']:
    qc_test = qc_singlet.copy()
    # Standardmessung entfernen (die letzten beiden measure-Befehle)
    # Die Schaltung qc_singlet hat am Ende 2 Mess-Operationen.
    qc_test.data = qc_test.data[:-2]
    # Sicherstellen, dass das klassische Register noch existiert
    qc_test.add_register(qc_test.cregs[0])

    # Alice misst in Z-Basis
    qc_test = measure_in_basis(qc_test, 0, 'Z')
    # Bob misst in 'Z' oder 'X'-Basis
    qc_test = measure_in_basis(qc_test, 1, basis_bob)

    # Führe Messung durch
    result = execute(qc_test, qasm_simulator, shots=shots).result()
    counts = result.get_counts(qc_test)
    counts_all.append((basis_bob, counts))

for basis, c in counts_all:
    print(f"Basis Bob: {basis}, Ergebnis: {c}")

- In einem realen **E91-Protokoll** würden Alice und Bob beide **zufällig** ihre Basen wählen und dann **öffentlich** vergleichen, wer in welchen Basen gemessen hat (ohne die konkreten Messresultate zu verraten).  
- Nur Messungen, bei denen beide die **gleiche** Basis gewählt haben, werden für den finalen Schlüssel verwendet (wenn keine Störung festgestellt wurde).

---

### 3.3. Korrelation und CHSH-Ungleichung
Ein Teil der Messungen wird benutzt, um zu überprüfen, ob die **CHSH-Ungleichung** verletzt wird. Das bestätigt die **Nicht-Lokalität** und damit die Sicherheit.

#### Die CHSH-Ungleichung
Sie prüft die Korrelationen von vier verschiedenen Mess-Einstellungen ($A, A'$, $B, B'$).  
Der CHSH-Ausdruck ist:
$$ S = E(A, B) - E(A, B') + E(A', B) + E(A', B'), $$
wobei nach klassisch-lokalem Realismus $|S| \le 2$ gelten muss, während die Quantenmechanik einen Wert bis $2\sqrt{2}$ erlauben kann (sogenannte Tsirelson-Grenze).

#### Wie misst man die Korrelation?
Wir ordnen den Messresultaten $\pm 1$ zu und berechnen:
$$E(A, B) = \langle a \cdot b \rangle = P(a=b) - P(a\neq b).$$

**Beispielcode für die Berechnung von Korrelationen (CHSH)**:

In [ ]:
def calc_expectation(counts, shots):
    """
    Erwartungswert E = (N(0,0) + N(1,1) - N(0,1) - N(1,0)) / shots
    Dabei wird für Qubits: 0 => +1, 1 => -1 angesetzt.
    """
    # counts sieht z.B. so aus: {'00': 123, '01': 200, '10': 210, '11': 467}
    # In Qiskit ist die Reihenfolge der Bits i.d.R. (qubit N-1 ... qubit 0),
    # je nachdem, wie gemessen wird.
    
    n_00 = counts.get('00', 0)
    n_01 = counts.get('01', 0)
    n_10 = counts.get('10', 0)
    n_11 = counts.get('11', 0)

    return (n_00 + n_11 - n_01 - n_10) / shots

# Beispiel: Wir messen vier unterschiedliche Einstellungen
# (A, B), (A, B'), (A', B), (A', B') und berechnen S
# Zur Vereinfachung: A=Z, A'=X, B=Z, B'=X
bases = [('Z','Z'), ('Z','X'), ('X','Z'), ('X','X')]
shots = 1024

S_components = []

for (basis_a, basis_b) in bases:
    qc_chsh = qc_singlet.copy()
    # Alte Messungen entfernen
    qc_chsh.data = qc_chsh.data[:-2]
    # Neue Messungen
    qc_chsh = measure_in_basis(qc_chsh, 0, basis_a)
    qc_chsh = measure_in_basis(qc_chsh, 1, basis_b)

    result_chsh = execute(qc_chsh, qasm_simulator, shots=shots).result()
    counts_chsh = result_chsh.get_counts(qc_chsh)
    E_ab = calc_expectation(counts_chsh, shots)
    S_components.append(E_ab)

S = S_components[0] - S_components[1] + S_components[2] + S_components[3]
print("S:", S)

Wenn $|S| > 2$, ist die **CHSH-Ungleichung verletzt** – ein Beweis für **Nicht-Lokalität**.  
In idealen Simulationen (ohne Rauschen) kann man sich Werten bis $2\sqrt{2} \approx 2.828$ nähern.

---
### 3.4. Schlüsselerzeugung
Der nächste Schritt im E91-Protokoll besteht darin, dass:
1. Alice und Bob **öffentlich** (über einen klassischen Kanal) die Basen austauschen, in denen sie gemessen haben.  
2. Sie werfen alle Messwerte weg, bei denen sie **verschiedene** Basen verwendet haben (bzw. behalten genau die Paare für die Schlüsselerzeugung, die in derselben Basis gemessen wurden).  
3. Sie führen ggf. **Fehlerkorrektur** (Information Reconciliation) und **Privacy Amplification** durch, um einen **sicheren Schlüssel** zu erhalten.  
4. Die Verletzung der **CHSH-Ungleichung** in einer Teilmenge der Messungen dient als **Sicherheits-Garantie**.

In einem reinen Coding-Beispiel würde man nun systematisch die Messresultate in denselben Basen filtern und daraus eine Bitfolge ableiten.

---

## 4. Unterschiede zum BB84 Algorithmus
1. **Art der Zustände**  
   - **BB84**: Präpariert einzelne Qubits (Photonen) in unterschiedlichen Basen (z. B. Z und X).  
   - **E91**: Verwendet **verschlüsselte (verschränkte) Teilchenpaare** und überprüft deren Korrelation durch Bell-Tests (CHSH).  

2. **Erkennung von Abhörern**  
   - **BB84**: Stellt Störungen fest, indem man eine Testmenge in bekannter Basis misst und die Fehlerrate bestimmt.  
   - **E91**: Überprüft explizit die **Verletzung** der Bell-/CHSH-Ungleichung, um auf Abhöraktivitäten zu schließen.  

3. **Praktische Umsetzung**  
   - **BB84**: Etwas einfacher in der experimentellen Umsetzung (man braucht „nur“ Einzelphotonquellen und -detektoren).  
   - **E91**: Benötigt **Verschränkungsquellen**, ist aufwendiger, hat aber eine besonders elegante theoretische Begründung (direkt aus der Nicht-Lokalität).

---

## 5. Was bedeutet das in der „Realität“?
- **Photonen** werden oft via **Spontane Parametrische Fluoreszenz** (SPDC) in Kristallen erzeugt – so entstehen verschränkte Photonenpaare.  
- **Alice** und **Bob** sind räumlich getrennt (z. B. 100m oder viele Kilometer durch Glasfaserleitungen), jedes Photon reist zu einer Messstation.  
- Die Messungen geschehen dort mit **Polarisationsfiltern** oder **phasenschiebenden Elementen**, die verschiedene Basis-Einstellungen abbilden.  
- Treten **Verluste** und **Rauschen** auf, verringert sich die gemessene **CHSH-Verletzung**.  
- Liegt sie dennoch signifikant über dem klassischen Grenzwert (2), so gilt das System noch als sicher.

---

## 6. Fazit
Mit dem **E91-Protokoll** wird deutlich, wie man mithilfe von **Verschränkung** und **CHSH-Tests** einen sicheren quantenmechanischen Schlüssel verteilen kann. Die Sicherheit beruht auf der **Nicht-Lokalität** der Quantenmechanik: Ein Angreifer kann nicht ohne Störung in das System eingreifen, da sonst die charakteristische **CHSH-Verletzung** verschwindet.

BB84 und E91 sind die beiden **bekanntesten** QKD-Protokolle. Während BB84 meist einfacher umzusetzen ist, besitzt E91 durch seine direkte Verwendung von **Bell-Tests** einen ganz besonderen, fundamentalen Charme in der Quantenkryptographie.

**Weiterführende Übung**:
- Probiere aus, verschiedene Rotationen (z. B. Y-Basis, d. h. ein S-Gate gefolgt von einem H-Gate zur Messung) in Qiskit zu implementieren.  
- Berechne den CHSH-Wert für unterschiedliche Einstellungen.  
- Versuche, dich an die theoretisch maximal mögliche Verletzung $|S| = 2\sqrt{2}$ heranzutasten.

---
**Ende des interaktiven Tutorials**